# Tensorflow Fruits360-VGG16 Patch Demo

This notebook demonstrates the adversarial patch attack applied on the VGG16 model, as well as adversarial training defenses.

The following two sections cover experiment setup and is similar across all demos.

## Setup: Experiment Name and Fruits360 Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

### Important: Users will need to verify or update the following parameters:
- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the Fruits360 dataset directory. Currently set to `/nfs/data/Fruits360-Kaggle-2019/fruits-360` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Fruits360-Kaggle-2019/fruits-360"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_fruits360_adversarial_patches"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [2]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

Available rules:

clean               Remove temporary files 
data                Download and prepare MNIST dataset 
initdb              Initialize the RESTful API database 
services            Launch the Minio S3 and MLFlow Tracking services 
teardown            Destroy service containers 
workflows           Create workflows tarball 


### Fruits360 Dataset Overview:
The training and testing images in this directory are saved as JPEG files and are organized into the following folder structure:
    
    fruits360
    ├── Test
    │   ├── Apple Braeburn
    │   ├── Apple Crimson Snow
    │   ├── Apple Golden 1
    │   ├── Apple Golden 2
    │   ├── Apple Golden 3
    │   ├── ...
    │   └── Walnut
    └── Training
    │   ├── Apple Braeburn
    │   ├── Apple Crimson Snow
    │   ├── Apple Golden 1
    │   ├── Apple Golden 2
    │   ├── Apple Golden 3
    │   ├── ...
    │   └── Walnut
  
  
  The subfolders under `fruits360/Training/` and `fruits360/Test` are the classification labels for the images in the dataset.
  There are 120 labels or subfolders for the training and test sets.

This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Setup: REST API and Source Code

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [3]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [4]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [5]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'createdOn': '2020-11-05T09:37:19.652250',
 'experimentId': 11,
 'lastModified': '2020-11-05T09:37:19.652250',
 'name': 'howard_fruits360_adversarial_patches'}

### The following sections will now cover the adversarial patch demo.






# Adversarial Patches: Baseline Training

Now, we will train our baseline VGG16 model on the Fruits360 dataset. 
We will be submitting our jobs to the `"tensorflow_gpu"` queue.

Once the experiment is finished, we will examine the accuracy results of our model.

In [6]:
response_vgg16_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        "-P register_model=True",
        "-P model_architecture=vgg16",
        "-P epochs=30",
        f"-P data_dir_train={DATASET_DIR}/Training",
        f"-P data_dir_test={DATASET_DIR}/Test",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for VGG16 neural network submitted")
print("")
pprint.pprint(response_vgg16_train)

Training job for VGG16 neural network submitted

{'createdOn': '2021-02-14T08:41:36.221163',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=20 -P register_model=True -P '
                     'model_architecture=vgg16 -P epochs=30 -P '
                     'data_dir_train=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P '
                     'data_dir_test=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test',
 'experimentId': 11,
 'jobId': '1926a003-02ee-4c42-b6e1-98632c2b1754',
 'lastModified': '2021-02-14T08:41:36.221163',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/a4d0b3f056c84737acc5ec3399293a0b/workflows.tar.gz'}


In [7]:
# The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
# The run ID is needed to link dependencies between jobs.

def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]

def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response

def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response

    

In [8]:
# Now wait for the job to complete before proceeding to next steps.

response_vgg16_train = wait_until_finished(response_vgg16_train)
print("Training job for VGG16 neural network")
pprint.pprint(response_vgg16_train)

Training job for VGG16 neural network
{'createdOn': '2021-02-14T08:41:36.221163',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=20 -P register_model=True -P '
                     'model_architecture=vgg16 -P epochs=30 -P '
                     'data_dir_train=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P '
                     'data_dir_test=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test',
 'experimentId': 11,
 'jobId': '1926a003-02ee-4c42-b6e1-98632c2b1754',
 'lastModified': '2021-02-14T09:08:17.050865',
 'mlflowRunId': '985d7aaf6da242f79704de004fb9f102',
 'queueId': 2,
 'status': 'finished',
 'timeout': '1h',
 'workflowUri': 's3://workflow/a4d0b3f056c84737acc5ec3399293a0b/workflows.tar.gz'}


## Checking baseline VGG16 job accuracy.

Once the job has finished running we can view the results either through the MLflow URI or by accessing the job via MLflow client.

Here we will show the baseline accuracy results from the previous training job.

Please see [Querying the MLFlow Tracking Service](#querying_cell) section for more details. 

In [9]:
# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run
        
results = get_mlflow_results(response_vgg16_train)
pprint.pprint(results.data.metrics)

{'accuracy': 0.9999380707740784,
 'auc': 1.0,
 'fn': 3.0,
 'fp': 3.0,
 'loss': 0.00020960499134656813,
 'precision': 0.9999380707740784,
 'recall': 0.9999380707740784,
 'restored_epoch': 6.0,
 'stopped_epoch': 11.0,
 'tn': 5763286.0,
 'tp': 48428.0,
 'training_time_in_minutes': 25.5569761,
 'val_accuracy': 0.9912985563278198,
 'val_auc': 0.9986258149147034,
 'val_fn': 108.0,
 'val_fp': 102.0,
 'val_loss': 0.043770697606182296,
 'val_precision': 0.9915429949760437,
 'val_recall': 0.9910499453544617,
 'val_tn': 1435871.0,
 'val_tp': 11959.0}


## Deploying and Testing Adversarial Patches

Now we will create and apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

We will also apply the patches over the training set for the adversarial training defense evaluation.

### Patch Generation:
The following job will generate the adversarial patches. 

Feel free to adjust the input parameters to see how they impact the effectiveness of the patch attack.

In [10]:
# Create Patches
response_vgg16_patches = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_vgg16/1",
            "-P model_architecture=vgg16",
            f"-P data_dir={DATASET_DIR}/Training",
            "-P num_patch_gen_samples=20",
            "-P num_patch=1",
            "-P patch_target=5"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_train["jobId"],
)

print("Patch attack (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_vgg16_patches)
print("")

response_vgg16_patches = get_run_id(response_vgg16_patches)

Patch attack (VGG16 architecture) job submitted

{'createdOn': '2021-02-14T09:08:17.705333',
 'dependsOn': '1926a003-02ee-4c42-b6e1-98632c2b1754',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P model=howard_fruits360_adversarial_patches_vgg16/1 -P '
                     'model_architecture=vgg16 -P '
                     'data_dir=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P num_patch_gen_samples=20 -P num_patch=1 -P '
                     'patch_target=5',
 'experimentId': 11,
 'jobId': '325d0a68-10ad-4843-9122-a450519524da',
 'lastModified': '2021-02-14T09:08:17.705333',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/a23aa3e7e6da49518ff971308c56e260/workflows.tar.gz'}



### Deploying and Testing Adversarial Patches:

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [11]:
# Deploy Patch attack on training set.
response_deploy_vgg16_patches_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_vgg16_patches['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_vgg16/1",
            "-P model_architecture=vgg16",
            f"-P data_dir={DATASET_DIR}/Training",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_patches["jobId"],
)

print("Patch deployment (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_deploy_vgg16_patches_training)
print("")

response_deploy_vgg16_patches_training = get_run_id(response_deploy_vgg16_patches_training)

    
# Deploy Patch attack on test set.
response_deploy_vgg16_patches_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_vgg16_patches['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_vgg16/1",
            "-P model_architecture=vgg16",
            f"-P data_dir={DATASET_DIR}/Test",
            "-P patch_deployment_method=corrupt"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_patches["jobId"],
)

print("Patch deployment (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_deploy_vgg16_patches_testing)
print("")

response_deploy_vgg16_patches_testing = get_run_id(response_deploy_vgg16_patches_testing)
    

Patch deployment (VGG16 architecture) job submitted

{'createdOn': '2021-02-14T09:08:21.826606',
 'dependsOn': '325d0a68-10ad-4843-9122-a450519524da',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=8106d9ea17a54c3cb3e72bbd9702c9c9 -P '
                     'model=howard_fruits360_adversarial_patches_vgg16/1 -P '
                     'model_architecture=vgg16 -P '
                     'data_dir=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training',
 'experimentId': 11,
 'jobId': 'af05a67f-e7ba-4dbe-94b1-da4f9a97fc47',
 'lastModified': '2021-02-14T09:08:21.826606',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/890b0d89c9dd4734a0bd3a1704f98c19/workflows.tar.gz'}

Patch deployment (VGG16 architecture) job submitted

{'createdOn': '2021-02-14T09:21:36.767584',
 'dependsOn': '325d0a68-10ad-4843-9122-a450519524da',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=8106d9ea17a54c3cb3e72bbd9702c9c9 -P '
  

## Patch Attack Evaluation: Baseline VGG16 Model
Now we will run an inference step to check the patch-attacked dataset with our VGG16-trained model.

In [12]:
# Check patched dataset results   
response_infer_vgg16_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_vgg16/1",
            "-P batch_size=512",
            "-P model_architecture=vgg16",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_vgg16_patches_testing["jobId"],
)

print("Patch evaluation (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_infer_vgg16_patch)
print("")

Patch evaluation (VGG16 architecture) job submitted

{'createdOn': '2021-02-14T09:21:40.892260',
 'dependsOn': '8b7a3ffb-8f41-4a3d-82c6-ded2c7e63d1c',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=6646fe2f2e4e46c6a031a162d703559b -P '
                     'model=howard_fruits360_adversarial_patches_vgg16/1 -P '
                     'batch_size=512 -P model_architecture=vgg16 -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset',
 'experimentId': 11,
 'jobId': '1d756b95-3213-4f67-bd22-c2c5d3895f4d',
 'lastModified': '2021-02-14T09:21:40.892260',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/1ee0dce953ef4defa270ffc7c5de91ba/workflows.tar.gz'}



In [13]:
# Wait for the job to finish
response_infer_vgg16_patch = wait_until_finished(response_infer_vgg16_patch)

# Check on the patch evaluation results
results = get_mlflow_results(response_infer_vgg16_patch)
print("Baseline model results on adversarially patched dataset: ")
pprint.pprint(results.data.metrics)

Baseline model results on adversarially patched dataset: 
{'accuracy': 0.7092876434326172,
 'auc': 0.9357548952102661,
 'fn': 6226.0,
 'fp': 5297.0,
 'loss': 1.7336138808486625,
 'precision': 0.7308298349380493,
 'recall': 0.6978843212127686,
 'tn': 2447055.0,
 'tp': 14382.0}



We can see that the adversarial patch attack causes a noticeable decrease in the model's accuracy scores.

We will now test various defenses against the patch attacked images.

# Defense: Adversarial Training

The next part of the adversarial patch demo focuses on investigating effective defenses against the attack.

### Adversarial Training Defense

We will train a new copy of the VGG16 model on training set that contains adversarial patches.

In doing so, the model learns to ignore the adversarial patches.

In [14]:
response_patches_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            f"-P testing_dataset_run_id={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P training_dataset_run_id={response_deploy_vgg16_patches_training['mlflowRunId']}",
            "-P batch_size=256",
            "-P register_model=True",
            "-P model_architecture=vgg16",
            "-P model_tag=adversarial_patch",
            "-P epochs=10",
            f"-P data_dir_train={DATASET_DIR}/Training",
            f"-P data_dir_test={DATASET_DIR}/Test",
            "-P load_dataset_from_mlruns=True",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_vgg16_patches_training["jobId"],
)

print("Patch adversarial training (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_patches_adv_training)
print("")

response_patches_adv_training = get_run_id(response_patches_adv_training)

Patch adversarial training (VGG16 architecture) job submitted

{'createdOn': '2021-02-14T09:54:51.952135',
 'dependsOn': 'af05a67f-e7ba-4dbe-94b1-da4f9a97fc47',
 'entryPoint': 'train',
 'entryPointKwargs': '-P '
                     'testing_dataset_run_id=6646fe2f2e4e46c6a031a162d703559b '
                     '-P '
                     'training_dataset_run_id=1e2f8da010d4430c8233aae74da93ad8 '
                     '-P batch_size=256 -P register_model=True -P '
                     'model_architecture=vgg16 -P model_tag=adversarial_patch '
                     '-P epochs=10 -P '
                     'data_dir_train=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P '
                     'data_dir_test=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test '
                     '-P load_dataset_from_mlruns=True',
 'experimentId': 11,
 'jobId': '645fdc4e-c66b-4b35-82ed-af3e40138714',
 'lastModified': '2021-02-14T09:54:51.952135',
 'mlflowRunId': None,
 'queueId':

In [15]:
response_evaluate_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_adversarial_patch_vgg16/None",
            "-P model_architecture=vgg16",
            "-P batch_size=256",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_patches_adv_training["jobId"],
)

print("Patch evaluation job submitted")
print("")
pprint.pprint(response_evaluate_adv_training)
print("")

response_patches_adv_training= wait_until_finished(response_patches_adv_training)
results = get_mlflow_results(response_patches_adv_training)
print("Adversarial Training Results:")
pprint.pprint(results.data.metrics)

Patch evaluation job submitted

{'createdOn': '2021-02-14T11:00:42.315694',
 'dependsOn': '645fdc4e-c66b-4b35-82ed-af3e40138714',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=6646fe2f2e4e46c6a031a162d703559b -P '
                     'model=howard_fruits360_adversarial_patches_adversarial_patch_vgg16/None '
                     '-P model_architecture=vgg16 -P batch_size=256 -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset',
 'experimentId': 11,
 'jobId': 'dcd71d42-e191-42c7-9790-056e25243ac6',
 'lastModified': '2021-02-14T11:00:42.315694',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/3a866351cd534cc6a03751435bc22d4f/workflows.tar.gz'}

Adversarial Training Results:
{'accuracy': 0.9994319081306458,
 'auc': 0.9999896287918091,
 'fn': 57.0,
 'fp': 52.0,
 'loss': 0.002543702680072632,
 'precision': 0.9994628429412842,
 'recall': 0.9

<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [16]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the VGG16 architecture,

In [17]:
run_adv_patches = mlflow_client.get_run(response_patches_adv_training["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [18]:
pprint.pprint(run_adv_patches.data.metrics)

{'accuracy': 0.9994319081306458,
 'auc': 0.9999896287918091,
 'fn': 57.0,
 'fp': 52.0,
 'loss': 0.002543702680072632,
 'precision': 0.9994628429412842,
 'recall': 0.9994112253189087,
 'restored_epoch': 2.0,
 'stopped_epoch': 7.0,
 'tn': 11520576.0,
 'tp': 96755.0,
 'training_time_in_minutes': 37.3107532,
 'val_accuracy': 0.993457019329071,
 'val_auc': 0.9999362230300903,
 'val_fn': 180.0,
 'val_fp': 130.0,
 'val_loss': 0.02069376747270948,
 'val_precision': 0.9946053624153137,
 'val_recall': 0.9925459623336792,
 'val_tn': 2873482.0,
 'val_tp': 23968.0}


To review the run's parameters, we use,

In [19]:
pprint.pprint(run_adv_patches.data.params)

{'baseline': 'None',
 'batch_size': '256',
 'data_dir_test': '/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test',
 'data_dir_train': '/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training',
 'dataset_seed': '1262593126',
 'entry_point_seed': '180331070667909949252666387562304663912',
 'epochs': '10',
 'learning_rate': '0.001',
 'load_dataset_from_mlruns': 'True',
 'min_delta': '-0.01',
 'model_architecture': 'vgg16',
 'model_tag': 'adversarial_patch',
 'monitor': 'val_loss',
 'opt_amsgrad': 'False',
 'opt_beta_1': '0.9',
 'opt_beta_2': '0.999',
 'opt_decay': '0.0',
 'opt_epsilon': '1e-07',
 'opt_learning_rate': '0.001',
 'opt_name': 'Adam',
 'optimizer': 'adam',
 'patience': '5',
 'register_model': 'True',
 'restore_best_weights': 'True',
 'seed': '-1',
 'tensorflow_global_seed': '481123530',
 'testing_dataset_run_id': '6646fe2f2e4e46c6a031a162d703559b',
 'training_dataset_run_id': '1e2f8da010d4430c8233aae74da93ad8',
 'validation_split': '0.2'}


To review the run's tags, we use,

In [20]:
pprint.pprint(run_adv_patches.data.tags)

{'mlflow.log-model.history': '[{"run_id": "45638a45eaae473badca2179fd3f9815", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2021-02-14 11:39:06.843447", "flavors": '
                             '{"keras": {"keras_module": '
                             '"tensorflow_core.keras", "keras_version": '
                             '"2.2.4-tf", "data": "data"}, "python_function": '
                             '{"loader_module": "mlflow.keras", '
                             '"python_version": "3.7.9", "data": "data", '
                             '"env": "conda.yaml"}}}]',
 'mlflow.project.backend': 'securingai',
 'mlflow.project.entryPoint': 'train',
 'mlflow.source.name': '/work/tmpkm3a6l4t',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'securingai',
 'securingai.dependsOn': 'af05a67f-e7ba-4dbe-94b1-da4f9a97fc47',
 'securingai.jobId': '645fdc4e-c66b-4b35-82ed-af3e40138714',
 'securingai.queue': 'tensorflow_gpu'}


There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).